# Zapytania dot. metadanych 
W pracy z danymi często zachodzi potrzeba, aby sprawdzić w jaki sposób przechowywane są dane (typy kolumn w tabeli, partycje, format plików, dostępne bazy danych). Aby ułatwić nam to zadanie zapoznamy się z odpowiednimi poleceniami.

Polecenia dot. baz:

1. `show databases` - wyświetla dostępne bazy
2. `describe database` nazwa_bazy - opis bazy o nazwie nazwa_bazy
3. `create database` nazwa_bazu - tworzenie nawej bazy o nazwie nazwa_bazy
4. `drop database` nazwa_bazy - usuwanie bazy o nazwie nazwa_bazy

W następnych komórkach można samodzielnie zapoznać się z wynikami tych funkcji

In [0]:
show databases

In [0]:
describe database default

In [0]:
create database test;

In [0]:
show databases;

In [0]:
drop database test;

Polecenia dot. tabel:
1. `show tables` [in nazwa_bazy] - wyświetla tabele w domyślnie wybranej bazie (opcjonalnie po `in` można podać inną bazę)
2. `show create table` nazwa_tabeli - wyświetla skrypt tworzący tabelę nazwa_tabeli
3. `show columns in` nazwa_tabeli - wyświetla kolumny w tabeli nazwa_tabeli
4. `show partitions` nazwa_tabeli - wyświetla partycje w tabeli partycjonowanej
5. `show tblproperties` - wyświetla właściwości tabeli
6. `describe table` - wyświetla opis tabeli

W następnych komórkach można samodzielnie zapoznać się z wynikami tych funkcji

In [0]:
show tables

In [0]:
show tables in default

In [0]:
show create table uam_categories;

Możemy skorzystać z nazwy schematu, jeśli tabela znajduje się w innym schemacie niż domyślny.

In [0]:
show create table default.uam_categories;

In [0]:
show columns in uam_categories;

Żadna z tabel nie jest partycjonowana, więc poniższe polecenie zakończy się błędem

In [0]:
show partitions uam_categories

In [0]:
show tblproperties uam_categories

In [0]:
describe uam_categories

`desc` jest aliasem `describe`

In [0]:
desc uam_categories

Teraz możemy zapoznać się, co zawierają tabele umieszczone w bazie *default*. Do dyspozycji mamy uproszczone schematy 3 tabel opisujące podstawowe byty serwisu aukcyjnego:
1. *uam_categories* - kategorie, w których znajdują się oferty (*uam_offers*)
2. *uam_offers* - dostępne oferty
3. *uam_orders* - złożone zamówienia

Znaczenie poszczególnych kolumn będzie wyjaśnione w trakcie następnych części kursu

In [0]:
desc uam_categories

In [0]:
desc uam_offers

In [0]:
desc uam_orders

# Klauzula SELECT

Za pomocą polecenie `SELECT` będziemy odpytywać nasze tabele. Składnia polecenie Spark SQL nie różni się znacząco od innych dialektów SQL:

1. `SELECT`  - projekcja, czyli wybranie kolumn, które ma zwracać zapytanie
2. `FROM` - wskazanie relacji, z której odczytujemy dane
3. `WHERE` - selekcja, czyli wybór odpowiednich wierszy
4. `GROUP BY` - grupowanie
5. `ORDER BY`/`SORT BY` - sortowanie (globalne)/sortowanie wewnątrz partycji danych
6. `DISTRIBUTE BY` - repartycjonowanie po liście kolumn
7. `CLUSTER BY` - skrót od DISTRIBUTE BY oraz SORT BY
8. `WINDOW`  - funkcje def. okno (tzw. funkcje analityczne)
9. `LIMIT` - ograniczenia liczby zwracanych wierszy

W czasie zajęć pominiemy klauzule: `SORT BY`, `DISTRIBUTE BY` i `CLUSTER BY`.

Docs: https://docs.gcp.databricks.com/sql/language-manual/sql-ref-syntax-qry-select.html

## Projekcja i selekcja - prosty przykład

Znajdźmy wszystkie oferty (identyfikator, ilość oraz łączną cenę), których sprzedano więcej niż 10.

Identyfikatorem jest kolumna *offer_id*, ilość znajduje się w kolumnie *quantity*, cena jednostkowa w kolumnie *unit_price*, zaś łączna cena w kolumnie *price* (*quantity* * *unit_price*)

In [0]:
select
  offer_id,
  quantity,
  price as total
from
  uam_orders
where
  quantity > 10;

Projekcja i selekcja - zadanie

Znajdź wszystkie daty transakcji (`uam_orders.buyingTime`), gdzie cena przekracza 200 zł (`uam_orders.price`)

Rozwiązanie jest w ukrytej kolumnie poniżej

In [0]:
select
  buyingTime
from
  uam_orders
where
  price > 200;

## Projekcja i selekcja - `lateral view`

Klauzula `lateral view` służy do umieszczania elementów struktur złożonych (tablica, mapa) w osobnych wierszach. 

Załóżmy, że chcielibyśmy elementy poniższej tablicy umieścić w osobnych wierszach:

In [0]:
select
  array(1, 2, 3, 4) as arr

Możemy skorzystać z klauzuli `lateral view` i funkcji `explode`:

In [0]:
select
  col
from
  lateral view explode(array(1, 2, 3, 4)) rel as col;

Lub wyłącznie z `explode`

In [0]:
select
  explode(array(1, 2, 3, 4))

`lateral view` - przykład

W tabeli *uam_offers* kolumna  *types* oznaczająca typ oferty jest typu *array < string >*

Aby sprawdzić, wszystkie możliwe typy ofert możemy posłużyć się zapytaniem

In [0]:
select
  distinct types
from
  uam_offers 

In [0]:
select
  distinct table_.col_
from
  uam_offers lateral view explode(types) table_ as col_

`lateral view` - zadanie

Znajdź wszystkie oferty (identyfikator *offer_id*, nazwę *offer_name*), które posiadają typ AUCTION. Spróbuj rozwiązać zadanie korzystając z `lateral view`, a następnie bez tej funkcji (użyj `array_contains`). Zbiór wynikowy ogranicz do 10 rekordów (klauzula `LIMIT`)

Podpowiedź: opis funkcji można sprawdzić poleceniem: `describe function array_contains`

Rozwiązania znajdują się poniżej

In [0]:
select
  offer_id,
  offer_name
from
  uam_offers lateral view explode(types) tab as col
where
  tab.col = 'AUCTION'
LIMIT
  10

In [0]:
select
  offer_id,
  offer_name
from
  uam_offers
where
  array_contains(types, 'AUCTION')
limit
  10;

## Funkcje wyższego rzędu (wyrażenia lambda)

Dla wersji Spark począwszy od 2.4.0 dostępne są funkcje wyższego rzędu (`higher-order functions`), które ułatwiają prace z typami kolumn array i map ([docs](https://docs.databricks.com/delta/data-transformation/higher-order-lambda-functions.html#)).

W przypadku, gdy chcielibyśmy zamienić wielkie litery na małe w *uam_offers.types*, możemy to zrobić następująco:

In [0]:
select
  offer_id,
  types as original_types,
  transform(types, x -> lower(x)) as lowercased_types
from
  uam_offers

Funkcja `transform` jako pierwszy argument przyjmuje tablicę a następnie funkcję anonimową, która określa rodzaj transformacji. W powyższym przypadku, każdy element tablicy jest przekształcany funkcją `lower`.


Możemy również posłużyć się klauzulami `lateral_view` i `group by` (`group by` dokładniej umówimy później), ale rozwiązanie będzie znacznie mniej zwięzłe:

In [0]:
select
  offer_id,
  collect_list(original_type) as original_types,
  collect_list(lowercased_type) as lowercased_types
from
  (
    select
      offer_id,
      type_column as original_type,
      lower(type_column) as lowercased_type
    from
      uam_offers lateral view explode(types) exploded_type as type_column
  )
group by
  offer_id

Przydatne są też funkcje `exists` i `filter`. `exists` może być użyta w klauzuli WHERE do sprawdzenia, czy jakikolwiek element w tablicy spełnia warunek wyrażony funkcją.

Przykład - `exists`
Policzymy oferty zawierające atrybut o nazwie (name) "rozmiar"

In [0]:
desc uam_offers;

In [0]:
select
  count(*) -- attributes
from
  uam_offers
where
  exists(attributes, a -> a.name = "rozmiar");

Funkcja `filter` umożliwia usunięcie zbędnych elementów z tablicy. Możemy ograniczyć wyświetlane atrybuty w ofertach do tych, które opisują stan:

In [0]:
select filter(attributes, a -> a.name = "stan") from uam_offers;

Możemy również łączyć funkcje `explode` oraz wyższego rzędu (np. `exists`).

Na przykład znajdźmy wszystkie oferty, które posiadają atrybut o nazwie "stan" i jedna z wartości tego atrybuty to "nowy". 

Poniższy kod jest mało czytelnej (wg mnie) i mało przypomina SQL, do którego jesteśmy przyzwyczajeni.

In [0]:
select
  distinct offer_id
from
  uam_offers LATERAL VIEW explode(attributes) t as attrib
WHERE
  attrib.name = "stan"
  and exists(attrib.`values`, v -> v = "nowy")

Ćwiczenie
Znajdź wszystkie oferty (identyfikator offer_id, nazwę offer_name), które posiadają typ AUCTION - tym razem używając `exists`

Rozwiązanie

In [0]:
select
  offer_id,
  offer_name
from
  uam_offers
where
  exists(types, t -> t = "AUCTION")

## Operacje złączenia

W następnej części zajmiemy się operacjami złączenia dostępnymi w Spark SQL:

1. `Inner join` - rekordy z obydwu łącząnych tabel muszą spełniać warunek połączeniowy
2. `Left outer join` - wszystkie rekordy z relacje po lewej stronie klauzuli są uwzględniane w zbiorze wynikowym i łączone są te z prawej strony, które spełniają warunek połączeniowy
3. `Right outer join` - wszystkie rekordy z relacje po prawej stronie klauzuli są uwzględniane w zbiorze wynikowym i łączone są te z lewej strony, które spełniają warunek połączeniowy
4. `Left semi join` - uwzględnia rekordy z lewej strony klauzuli, które spełniają warunek połączeniowy pomijając prawą tabelę w zbiorze wynikowym. Przy wielokrotnym złączeniu rekordów w zbiorze wynikowym umieszczany jest tylko jeden rekord
5. `Right semi join` - analogicznie jak `left semi join` z tym, że tabele zamienione są stronami
6. `Left anti join` - uwzględnia rekordy z lewej strony klauzuli, dla których nie istnieją łączące się rekordy z tabeli po prawej stronie
7. `Natural join` - złączenie, które bazuje na takich samych nazwach kolumn w obydwu tabelach
8. `Cross join` - iloczyn kartezjański tabel (brak warunków połączeniowych)

Operacja złączenia (`join`) - przykład `INNER JOIN`, `NATURAL JOIN` i `LEFT SEMI JOIN`

Sporządź zestawienie ofert sprzedanych danego dnia (identyfikator, nazwa). Wybierz tylko te oferty, które są droższe niż 200 zł.

Uwagi:
1. Warto zwrócić uwagę, że `SEMI JOIN` nie dubluje rekordów.
2. Proszę spróbować usunąć `DISTINCT` z przykładów poniżej i porównać wyniki.

In [0]:
select
  distinct o.offer_id,
  o.offer_name
from
  uam_offers o
  join uam_orders t on o.offer_id = t.offer_id
where
  o.buynow_price > 200;

In [0]:
select DISTINCT
  o.offer_id,
  o.offer_name
from
  uam_offers o
  natural join uam_orders
where
  o.buynow_price > 200;

In [0]:
select
  o.offer_id,
  o.offer_name
from
  uam_offers o 
  left semi join uam_orders t on o.offer_id = t.offer_id
where
  o.buynow_price > 200;

Operacja złączenia - Przykład `LEFT/RIGHT OUTER JOIN` i `LEFT ANTI JOIN`

Znajdźmy oferty tańsze niż 10 zł, które nie znalazły nabywcy?

In [0]:
select
  *
from
  uam_offers o
  left outer join uam_orders t on o.offer_id = t.offer_id
where
  t.offer_id is null
  and o.buynow_price < 10
limit
  10;

In [0]:
select
  o.offer_id,
  o.offer_name
from
  uam_orders t
  right outer join uam_offers o on o.offer_id = t.offer_id
where
  t.offer_id is null
  and o.buynow_price < 10
limit
  10;

In [0]:
select
  *
from
  uam_offers o 
  left anti join uam_orders t on o.offer_id = t.offer_id
where
  o.buynow_price < 10
limit
  10;

Operacja złączenia (join) - ćwiczenie


Czy istnieją kategorie bez ofert?

Przykładowe rozwiązanie

In [0]:
select
  count(*)
from
  uam_categories c left anti
  join uam_offers o on c.category_id = o.category_leaf
limit
  100;

## Grupowanie `group by`

Klauzula działa analogicznie jak w innych dialektach SQL. 


Oto prosty przykład:

Znajdź maksymalną i minimalną cenę (*buynow_price*) ofert w każdej z kategorii na 1. poziomie (*category_level1*)

In [0]:
select
  c.category_level1,
  min(cast(buynow_price as double)),
  max(cast(buynow_price as double))
from
  uam_offers o
  join uam_categories c on c.category_id = o.category_leaf
group by
  c.category_level1;

Przejdźmy do bardziej zaawansowanego przykładu z wykorzystaniem klauzul `with rollup` oraz `grouping sets`:

Znajdźmy maksymalną i minimalną cenę ofert w każdej z kategorii na 1. poziomie oraz na 1. i 2. poziomie kategorii oraz najtańszą i najdroższą ofertę. Posortujmy wyniki tak, aby podsumowania poszczególnych kategorii były na początku (zaczynając od globalnego podsumowania).

Oto dwie propozycje rozwiązań:

In [0]:
select
  c.category_level1,
  c.category_level2,
  min(cast(buynow_price as double)),
  max(cast(buynow_price as double))
from
  uam_offers o
  join uam_categories c on c.category_id = o.category_leaf
group by
  c.category_level1,
  c.category_level2 with rollup
order by
  category_level1 nulls first,
  category_level2 nulls first;

In [0]:
select
  c.category_level1,
  c.category_level2,
  min(cast(buynow_price as double)),
  max(cast(buynow_price as double))
from
  uam_offers o
  join uam_categories c on c.category_id = o.category_leaf
group by
  c.category_level1,
  c.category_level2 grouping sets(
    (),
    (c.category_level1),
    (c.category_level1, c.category_level2)
  )
order by
  category_level1 nulls first,
  category_level2 nulls first;

Klauzula `with rollup` pozwala na tworzenie podsumowań hierarchicznych na wszystkich poziomach zgodnie z kolejnością grupowania, natomiast `grouping sets` pozwala dowolnie tworzyć grupy podsumowań. Dostępna jest jeszcze klauzula `WITH CUBE`, ale tworzy ona wszystkie możliwe podsumowania, co nie ma sensu w naszym przypadku:

In [0]:
select
  c.category_level1,
  c.category_level2,
  min(cast(buynow_price as double)),
  max(cast(buynow_price as double))
from
  uam_offers o
  join uam_categories c on c.category_id = o.category_leaf
group by
  c.category_level1,
  c.category_level2 with cube
order by
  category_level1 nulls first,
  category_level2 nulls first;

Funkcje `grouping_id` pomaga określić jakiemu poziomu agregacji odpowiada dany rekord. Funkcja `grouping` określa, czy wartość `null` jest wynikiem agregacji, czy została wyliczona dla wartości `null` znajdujących się w zbiorze wejściowym. Najlepiej ilustruje to poniższy przykład.

In [0]:
select
  c.category_level1,
  c.category_level2,
  min(cast(buynow_price as double)),
  max(cast(buynow_price as double)),
  grouping_id(),
  grouping(c.category_level1),
  grouping(c.category_level2)
from
  uam_offers o
  join uam_categories c on c.category_id = o.category_leaf
group by
  c.category_level1,
  c.category_level2 with rollup
order by
  category_level1 nulls first,
  category_level2 nulls first;

Zadanie


Sporządź zestawienie obrotów (*double(unit_price)*  \*  *double(quantity)*) per kategoria na 1. poziomie (wraz z sumą całkowitą w ostatnim wierszu).

Rozwiązanie jest poniżej

In [0]:
select
  c.category_level1,
  sum(double(unit_price) * double(quantity)) as total
from
  uam_orders o
  join uam_categories c on c.category_id = o.category_id
group by
  c.category_level1 with rollup
order by
  category_level1 nulls last;

Do nakładania ograniczeń na wynik funkcji grupujących służy klauzula `HAVING`

Przykład

Sporządź zestawienie obrotów jak w powyższym zadaniu ograniczając zbiór wynikowy do kategorii z obrotami powyżej 2000 zł

In [0]:
select
  c.category_level1,
  sum(double(unit_price) * double(quantity)) as total
from
  uam_orders t
  join uam_categories c on c.category_id = t.category_id
group by
  c.category_level1 with rollup
having
  sum(double(unit_price) * double(quantity)) > 2000
order by
  category_level1 nulls last;

Warto zwrócić uwagę na kolejność wykonywania poszczególnych klauzul. Klauzula `HAVING` odfiltrowywuje po zgrupowaniu, obliczeniu sum cząstkowych i sumy globalnych obrotów (kolumna *null*).

Inne przydatne funkcje grupujące znajdują się poniżej:

`approxCount([distinct] …)`

`avg`

`collect_list`  

`collect_set`  

`corr`  

`count([distinct] ...) ` 

`first`  

`kurtosis`  

`last`  

`max`  

`mean`  

`min`  

`skewness`

`stddev_pop`  

`stddev_samp`  

`stddev`  

`sum([distinct] ...) `

`var_pop`  

`var_samp` 

`variance`

# Operacja `cache`


W przypadku, gdy wiemy, że będziemy wielokrotnie używali daną tabelę 
w zapytaniach możemy skorzystać z operacji cache. Spowoduje ona zachowanie danej tabeli w pamięci operacyjnej.

In [0]:
cache table uam_categories

Operacją powodującą zwolnienie pamięci podręcznej jest `uncache table`:

In [0]:
uncache table uam_categories;

Porównajmy plany zapytań przed i po operacji `cache`:

In [0]:
explain
select
  count(1)
from
  uam_categories;

In [0]:
cache table uam_categories;

In [0]:
explain
select
  count(1)
from
  uam_categories;

W planie zapytania przed operacją `cache` pojawia się:

*LocalTableScan [count(1) ...]*

co oznacza, że dane są odczytywane z dysku.


Po operacji `cache` w planie odnajdujemy:

*Scan In-memory table uam_categories*

Co oznacza, że dane są odczytywane z pamięci operacyjnej.

Całą pamięć podręczną możemy wyczyścić poleceniem `clear cache`:

In [0]:
clear cache

# Funkcje analityczne

Działanie funkcji analitycznych jest analogiczne jak w przypadku innych dialektów SQL.

Prześledźmy działanie funkcji analitycznych na przykładach

Znajdźmy kategorię (1. poziom), w której użytkownicy (*buyer_id*) dokonali 1. transakcji (jako kupujący). Wynik ogranicz do 10 rekordów.

In [0]:
select
  buyer_id,
  category_level1
from
  (
    select
      t.buyer_id,
      rank() over(
        partition by t.buyer_id
        order by
          t.buyingTime
      ) as rank_,
      t.category_id
    from
      uam_orders t
    qualify rank_ = 1
  ) t
  join uam_categories c on c.category_id = t.category_id
limit 10


Aby wywołać funkcję analityczną `rank()` musimy podać towarzyszącą jej definicję okna `over(partition by t.buyer_id order by t.buyingTime)`. W tym przypadku okno tworzy partycje dla każdego kupującego (*buyer_id*) i wymusza sortowanie po czasie zakupu (*buyingTime*). Zgodnie z czasem zakupu tworzony jest ranking (funkcja `rank`).

Alternatywne i bardziej zwięzłe rozwiązanie jest poniżej:

In [0]:
select
  distinct *
from
  (
    select
      FIRST_VALUE (category_level1) OVER (
        partition by buyer_id
        ORDER BY
          buyingTime
      ) first_cat_level_1,
      buyer_id
    from uam_categories c
    join uam_orders o on c.category_id = o.category_id
  )
limit 10

Dla zwiększenia czytelności kodu możemy się posłużyć klauzulą `WINDOW`:

In [0]:
select
  buyer_id,
  category_level1
from
  (
    select
      t.buyer_id,
      rank() over buying_time_window as rank_,
      t.category_id
    from
      uam_orders t 
    qualify rank_ = 1
    window buying_time_window as (
        partition by t.buyer_id
        order by
          t.buyingTime
      )
  ) t
  join uam_categories c on c.category_id = t.category_id
limit 10

Może być to szczególnie przydatne przy wykorzystywaniu tej samej definicji okna w jednym zapytaniu

Definicje okna mogą opierać się na sąsiednich wierszach (`rows between ... and ... `) lub na wartościach wierszy (`range between ... and ... `). 

W ten sposób możemy przykładowo:

Znaleźć maksymalną cenę sprzedanego towaru w odniesieniu do 2 poprzednich i następnych zakupów

In [0]:
select
  seller_id,
  max(price) over (
    partition by seller_id
    order by
      buyingTime rows between 2 preceding
      and 2 following
  ) as max_price_in_window
from
  uam_orders
limit
  10;

Znajdźmy liczbę ofert w zakresie cenowym między ceną danej oferty a ceną danej oferty + 5 PLN

In [0]:
select
  offer_id,
  count(*) over (
    order by
      cast(buynow_price as decimal(12, 2)) range between current row
      and 5.0 following
  ) as num_of_offers_with_the_same_price_or_higher_but_less_than_5PLN,
  cast(buynow_price as decimal(12, 2)) as buynow_price
from
  uam_offers
order by
  cast(buynow_price as decimal(12, 2)) nulls last
limit
  1000;

# Usuwanie i tworzenie tabel

Najważniejsze polecenia to:
1. `DROP TABLE [IF EXISTS] [nazwa_bazy.]nazwa_tabeli` - usuwa tabelę (z domyślnie ustawionej bazy). Jeśli tabela nie istnieje zwracany jest wyjątek. Wyjątek nie jest zwracany, gdy użyjemy opcjonalnej klauzuli `IF EXISTS`
2. `CREATE [TEMPORARY] TABLE [IF NOT EXISTS] [nazwa_bazy.]nazwa_tabeli ... ` - tworzy tabelę (opcjonalnie temporalną, czyli na potrzeby bieżącej sesji) o nazwie nazwa_tabeli i w bazie nazwa_bazy. Przykładowe schematy tabel można sprawdzić poleceniem `SHOW CREATE TABLE`. Nie będziemy szczegółowo analizować składni tego polecenia.
3. `CREATE [EXTERNAL] TABLE [IF NOT EXISTS] [nazwa_bazy.]nazwa_tabeli ... ` - podobne polecenie jak opisane w pkt. 2, ale służy do tworzenia tabel eksternalnych
4. `CREATE TABLE [IF NOT EXISTS] [nazwa_bazy.]nazwa_tabeli LIKE [nazwa_bazy_2.]nazwa_tabeli_2` - tworzy tabelę nazwa_tabeli o schemacie nazwa_tabeli_2 bez kopiowania danych

Dla zainteresowanych: https://docs.gcp.databricks.com/sql/language-manual/sql-ref-syntax-ddl-create-table.html

Tabele można również tworzyć korzystając ze składni kompatybilnej z Hive'em.

# Wstawianie i przechowywanie danych

Poniższe polecenia służą do wstawiania danych do tabeli [nazwa_bazy.]nazwa_tabeli (opcjonalnie konkretnej partycji specyfikacja_partycji) za pomocą klauzuli `SELECT`:
1. `INSERT INTO [TABLE] [nazwa_bazy.]nazwa_tabeli [PARTITION specyfikacja_partycji] SELECT ...` - powoduje dodanych nowych danych i zachowanie obecnych
2. `INSERT OVERWRITE TABLE [nazwa_bazy.]nazwa_tabeli [PARTITION specyfikacja_partycji] SELECT ...` - dodane nowe dane nadpisując dotychczasowe

Poniższe polecenia działają podobnie, ale umożliwiają samodzielnie specyfikowanie wstawianych rekordów:
1. `INSERT INTO [TABLE] [nazwa_bazy.]nazwa_tabeli [PARTITION specyfikacja_partycji] VALUES ...`
2. `INSERT OVERWRITE TABLE [nazwa_bazy.]nazwa_tabeli [PARTITION specyfikacja_partycji] VALUES ...`

Dostępne jest również polecenie `INSERT OVERWRITE [LOCAL] DIRECTORY ... `, które umożliwia zrzut rekordów bezpośrednio do systemu plików z pominięcięm tabel.

Przykłady

Utworzymy tabelę partycjonowaną i wstawimy rekord do konkretnej partycji:

In [0]:
drop table if exists uam_user_segments;

create table uam_user_segments (
  us_id bigint,
  trans_count bigint,
  turnover double,
  segment string
) partitioned by (month_ string) stored as orc;

insert into
  uam_user_segments partition(month_ = '2020-01-01')
values
  (1, 1, 1.0, 'small');
  
insert into
  uam_user_segments partition(month_ = '2020-02-01')
values
  (2, 100, 100.0, 'medium');

In [0]:
show create table uam_user_segments;

Sprawdzimy teraz w jakiej ścieżce zapisują się nasze dane na `DBFS` (szukamy atrybutu *Location*):

In [0]:
desc formatted uam_user_segments;

In [0]:
%fs ls dbfs:/user/hive/warehouse/uam_user_segments

In [0]:
%fs ls dbfs:/user/hive/warehouse/uam_user_segments/month_=2020-01-01/

Możemy odpytać naszą tabelę na dwa sposoby:

In [0]:
-- 1: SQL
select
  *
from
  uam_user_segments;

In [0]:
-- 2: bezpośrednio z katalogu
select
  *
from
  `orc`.`dbfs:/user/hive/warehouse/uam_user_segments/`

Nasza tabela jest partycjonowana, więc optymalniej będzie ją odpytywać zakładając warunek na klucz partycjonujący:

In [0]:
-- 1: SQL
select
  *
from
  uam_user_segments
where
  month_ = '2020-02-01';

In [0]:
-- 2: bezpośrednio z katalogu
select
  *
from
  `orc`.`dbfs:/user/hive/warehouse/uam_user_segments/month_=2020-02-01/`

Partycjonowanie jest jedną z najbardziej efektywnych i popularnych technik optymalizowania zapisu i odczytu danych. W przypadku zapisu możliwe są operacja dodawania, usuwania, zmiany ścieżki i nazwy partycji (szczegóły: https://docs.gcp.databricks.com/sql/language-manual/sql-ref-partition.html). Skupmy się jednak na samym odczycie i porównajmy plany zapytań z warunkiem na klucz partycji i bez:

In [0]:
explain
select
  *
from
  uam_user_segments
where
  month_ = '2020-02-01';

W powyższym planie interesujące są pogrubione fragmenty świadczące o odfiltrowaniu odpowiednich partycji na etapie wczytywania danych:

== Physical Plan == \*(1) ColumnarToRow +- FileScan orc default.uam_user_segments[us_id#6219L,trans_count#6220L,turnover#6221,segment#6222,month_#6223] Batched: true, DataFilters: [], Format: ORC, **Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/uam_user_segments/month_=2020-02-01]**, **PartitionFilters: [isnotnull(month_#6223), (month_#6223 = 2020-02-01)]**, PushedFilters: [], ReadSchema: struct<us_id:bigint,trans_count:bigint,turnover:double,segment:string>

Gdy nie założymy warunku na klucz partycji, to odczytywana jest cała tabela:

In [0]:
explain
select
  *
from
  uam_user_segments
where
  us_id = 1;

Dygresja - lokalny system plików (`local file system`) vs. rozproszony system plików (`distributed file system`)

Porównajmy poniższe polecenia. Pierwsze z nich odpytuje lokalny system plików. Gdyby to był notebook uruchamiany na naszym komputerze, to byłby to nasz dysk twardy. 

Drugie polecenie odpytuje rozproszony system plików.

In [0]:
%sh ls /

In [0]:
%fs ls /

Między danymi na rozproszonym systemie plików a metadanymi (nazwy schematów i tabel) istnieje zależność zdefiniowana w atrybucie `Location`, typie odczytywanego i zapisywanego pliku oraz konwencji nazewniczej (katalogi mapowane są na partycje).

W związku z tym nic nie stoi na przeszkodzie, aby odwrócić sposób naszego przetwarzania - najpierw utworzyć plik z danymi, a potem tabelę ekstarnalną (`EXTERNAL TABLE`):

In [0]:
INSERT
  OVERWRITE DIRECTORY '/tmp/sample_data' stored as parquet
select
  'Moje testowe dane' as testowa_kolumna

In [0]:
%fs ls /tmp/sample_data

In [0]:
create external table testowa_tabela (testowa_kolumna string) location '/tmp/sample_data' stored as parquet;

In [0]:
select * from testowa_tabela;

In [0]:
select * from `parquet`.`/tmp/sample_data`

Co się stanie z danymi jeśli usuniemy tabelę eksternalną:

In [0]:
DROP TABLE testowa_tabela;

In [0]:
%fs ls /tmp/sample_data

Musimy ręcznie usunąć dane:

In [0]:
%fs rm -r /tmp/sample_data

W przypadku dodawania nowych danych w tabelach partycjonowanych, dane będą zarejestrowane do odczytu po wykonaniu `MSCK REPAIR TABLE` (lub poleceniu `ALTER TABLE ... ADD PARTITION ... `):

In [0]:
msck repair table uam_user_segments;

Dygresja - dane możemy również zrzucić na lokalny system plików

In [0]:
%sh mkdir /tmp/dump

In [0]:
INSERT OVERWRITE LOCAL DIRECTORY '/tmp/dump' using csv select 'Moje testowe dane' as testowa_kolumna

In [0]:
%sh ls /tmp/dump

In [0]:
%sh cat /tmp/dump/*.csv

Zadanie

Na bazie tabeli *uam_categories*:
1. Stwórz zrzut do katalogu */tmp/orc_categories* na rozproszonym systemie plików w formacie orc
2. Stwórz nową bazę o nazwie *moja_baza*
3. Stwórz tabelę eksternalną o nazwie *moja_baza.orc_categories*, która będzie odczytywać */tmp/orc_categories*

Czy możemy stworzyć tabelę o innej strukturze?
Czy możemy odpytać taką tabelę?

In [0]:
show create table uam_categories;

Rozwiązanie

In [0]:
INSERT
  OVERWRITE DIRECTORY '/tmp/orc_categories' stored as orc
select
  *
from
  uam_categories;

CREATE DATABASE IF NOT EXISTS moja_baza;

DROP TABLE IF EXISTS moja_baza.orc_categories;

CREATE EXTERNAL TABLE moja_baza.orc_categories (
    `category_id` STRING,
    `category_level1` STRING,
    `category_level2` STRING,
    `category_level3` STRING
  ) STORED AS ORC LOCATION '/tmp/orc_categories';

SELECT
  count(1)
FROM
  moja_baza.orc_categories

Rozwiązanie - co się stanie gdy utworzymy tabelę o innej strukturze

In [0]:
drop table if exists moja_baza.orc_categories_2;

CREATE EXTERNAL TABLE moja_baza.orc_categories_2 (
  `category_id` STRING,
  `category_level1` STRING,
  `category_level2_xxx` STRING,
  `category_level3` STRING
) STORED AS ORC LOCATION '/tmp/orc_categories';

select * from moja_baza.orc_categories_2;

In [0]:
DROP TABLE IF EXISTS moja_baza.orc_categories_3;

CREATE EXTERNAL TABLE moja_baza.orc_categories_3 (
  `category_id` TIMESTAMP, -- <-- Inny typ danych
  `category_level1` INT -- <-- Inny typ danych
) STORED AS ORC LOCATION '/tmp/orc_categories';

select * from moja_baza.orc_categories_3;

Inne przykłady `CREATE` i `INSERT`


Utwórzymy tabele za pomocą poleceń `SELECT` i wykorzystaniem klauzuli `TABLESAMPLE` (służącej do zwracania próbki danych):

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/uam_orders_sample

In [0]:
create table if not exists uam_orders_sample as
select
  *
from
  uam_orders TABLESAMPLE (1 percent);
  
insert into
  uam_orders_sample
select
  *
from
  uam_orders TABLESAMPLE (10 rows);

Możemy też utworzyć tabelę bez danych:

In [0]:
drop table if exists moja_baza.orc_categories_sample;
create table if not exists moja_baza.orc_categories_sample like moja_baza.orc_categories;

# Operacje na zbiorach


Przeanalizujmy działania na zbiorach na przykładzie dwóch zapytań (odpowiednio liczby naturalne parzyste mniejsze od 10 i liczby naturalne mniejsze od 10). Wyjaśnienia działania funkcji `RANGE` znajduje się w kolejnej komórce.

In [0]:
select
  *
from
  range(0, 10, 2);


In [0]:
select
  *
from
  range(10);

Funkcja tabelaryczna `RANGE` służy do generowania tabel jednokolumnowych, gdzie kolejne wiersze są elementami ciągu arytmetycznego:

Funkcja obsługuje następujące parametry wejściowe: 

1. (end: long)
2. (start: long, end: long)
3. (start: long, end: long, step: long)
4. (start: long, end: long, step: long, numPartitions: integer)


start - 1. element ciągu

end - 2. element ostatni

step - określa o ile mają się zwiększać kolejne wiersze

numPartitions - liczba partycji

Sumę zbiorów otrzymamy (może zawierać duplikaty):

In [0]:
select
  *
from
  range(0, 10, 2)
union all
select
  *
from
  range(10);

Część wspólna zbiorów:

In [0]:
select
  *
from
  range(0, 10, 2)
intersect
select
  *
from
  range(10);

Różnica zbiorów:

In [0]:
select
  *
from
  range(10) 
  minus
select
  *
from
  range(0, 10, 2);

Zadanie

Znajdźmy oferty, które nie znalazły nabywcy (identyfikatory) korzystając z operacji na zbiorach.

Rozwiązanie jest poniżej:

In [0]:
select
  offer_id
from
  uam_offers minus
select
  offer_id
from
  uam_orders;